In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='SegNet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.51 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/8ce5535200b9419d9761e8ffb53b2882



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint, ArtPrintNoIntsect
from Model_Unet_github import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_no_intersect_1conv_64_channels_30epoch_unet_github_auto_dimension', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
parser.add_argument("-gpu", default='0', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=40, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/home/classify_copy/misc_test/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/home/classify_copy/misc_test/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtPrintNoIntsect(args.data_root, transform=transform_train)
        concat=arprint
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [ ]:
main()

The model is set to Training
Max training Iteration: 40
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /home/classify_copy/misc_test/datasets
Check point file: /home/classify_copy/misc_test/ckpt/segnet_no_intersect_1conv_64_channels_30epoch_unet_github_auto_dimension
Batch Size: 10
/home/classify_copy/misc_test/datasets/artifact_images_no_intersect already exists, skipping download
[0.05999042 0.94000958]
Instructions for updating:
Colocations handled automatically by placer.
GGG
[None, 32, 128, 1]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


0 conv1: (?, ?, ?, 32)
0 conv2: (?, ?, ?, 32)
1 conv1: (?, ?, ?, 64)
1 conv2: (?, ?, ?, 64)
2 conv1: (?, ?, ?, 128)
2 conv2: (?, ?, ?, 128)
1 h_deconv: (?, ?, ?, 64)
1 h_deconv_concat: (?, ?, ?, ?)
1 h_conv1_post_deconv: (?, ?, ?, 64)
1 h_conv2_post_deconv: (?, ?, ?, 64)
0 h_deconv: (?, ?, ?, 32)
0 h_deconv_concat: (?, ?, ?, ?)
0 h_conv1_post_deconv: (?, ?, ?, 32)
0 h_conv2_post_deconv: (?, ?, ?, 32)
0 outmap: (?, ?, ?, 2)
(?, ?, ?, 2)
loss: ()
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead.


...

VALID: Total accuracy: 96.447660%. Class 0 capture: 96.332241%. Class 1 capture: 98.256697%
Character error rate improved, save model
Epoch: 21  Training...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


TRAIN: Batch: 0.0 Loss: 0.008000555
accuracy = 0.969357
mean IU  = 0.814020
    class # 0 capture rate = 0.968343 
    class # 1 capture rate = 0.985081 
TRAIN: Batch: 0.003908998514580564 Loss: 0.0057376577
accuracy = 0.971850
mean IU  = 0.823378
    class # 0 capture rate = 0.970566 
    class # 1 capture rate = 0.992188 
TRAIN: Batch: 0.007817997029161129 Loss: 0.0044318093
accuracy = 0.974951
mean IU  = 0.838163
    class # 0 capture rate = 0.973625 
    class # 1 capture rate = 0.995898 
TRAIN: Batch: 0.011726995543741693 Loss: 0.009673904
accuracy = 0.959177
mean IU  = 0.779432
    class # 0 capture rate = 0.958231 
    class # 1 capture rate = 0.973108 
TRAIN: Batch: 0.015635994058322257 Loss: 0.011189746
accuracy = 0.951902
mean IU  = 0.774479
    class # 0 capture rate = 0.949798 
    class # 1 capture rate = 0.978225 
TRAIN: Batch: 0.019544992572902823 Loss: 0.0077578328
accuracy = 0.966772
mean IU  = 0.782543
    class # 0 capture rate = 0.965872 
    class # 1 capture rate 

TRAIN: Batch: 0.19154092721444765 Loss: 0.008635345
accuracy = 0.959396
mean IU  = 0.761630
    class # 0 capture rate = 0.957767 
    class # 1 capture rate = 0.988160 
TRAIN: Batch: 0.19544992572902822 Loss: 0.008667174
accuracy = 0.960130
mean IU  = 0.802478
    class # 0 capture rate = 0.958184 
    class # 1 capture rate = 0.984283 
TRAIN: Batch: 0.1993589242436088 Loss: 0.0072613857
accuracy = 0.965622
mean IU  = 0.776135
    class # 0 capture rate = 0.964638 
    class # 1 capture rate = 0.984352 
TRAIN: Batch: 0.20326792275818936 Loss: 0.0075411974
accuracy = 0.966917
mean IU  = 0.779349
    class # 0 capture rate = 0.965560 
    class # 1 capture rate = 0.993471 
TRAIN: Batch: 0.2071769212727699 Loss: 0.006844227
accuracy = 0.965501
mean IU  = 0.777652
    class # 0 capture rate = 0.964207 
    class # 1 capture rate = 0.989845 
TRAIN: Batch: 0.21108591978735047 Loss: 0.007218674
accuracy = 0.968887
mean IU  = 0.813934
    class # 0 capture rate = 0.967431 
    class # 1 captu

TRAIN: Batch: 0.3830818544288953 Loss: 0.006618553
accuracy = 0.973826
mean IU  = 0.815678
    class # 0 capture rate = 0.973609 
    class # 1 capture rate = 0.977809 
TRAIN: Batch: 0.3869908529434759 Loss: 0.004023981
accuracy = 0.981834
mean IU  = 0.841413
    class # 0 capture rate = 0.981448 
    class # 1 capture rate = 0.990385 
TRAIN: Batch: 0.39089985145805645 Loss: 0.010280685
accuracy = 0.949266
mean IU  = 0.750154
    class # 0 capture rate = 0.947329 
    class # 1 capture rate = 0.977324 
TRAIN: Batch: 0.394808849972637 Loss: 0.008304277
accuracy = 0.964278
mean IU  = 0.811471
    class # 0 capture rate = 0.963394 
    class # 1 capture rate = 0.975751 
TRAIN: Batch: 0.3987178484872176 Loss: 0.0050955294
accuracy = 0.974579
mean IU  = 0.814915
    class # 0 capture rate = 0.973655 
    class # 1 capture rate = 0.992519 
TRAIN: Batch: 0.4026268470017981 Loss: 0.0061336122
accuracy = 0.974922
mean IU  = 0.821887
    class # 0 capture rate = 0.974253 
    class # 1 capture r